In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from bff_processor.data_tools import regex_select, get_files, make_df
from bff_processor.bff_meta import preselection, band_cut, isin, identity, all_reg, jet_variations
from bff_processor.bff_cuts import *

from glob import glob
import re

In [ ]:
def one_in(list_of_strings, test_string):
    for string_in in list_of_strings:
        if string_in in test_string: return True
    return False

In [ ]:
from src.data_tools.get_file_list import get_file_df

In [ ]:
era = 2016
file_df = get_file_df()
file_df = file_df[file_df.era==era]
file_df.reset_index(inplace=True)

In [ ]:
from src.data_tools.apply_cuts import process_sample

In [ ]:
for i, row in file_df.iterrows():
    df = process_sample(row, verbose=True)
    if i==0: combined_df = df
    else: combined_df = pd.concat([combined_df, df])

In [ ]:
np.unique(df.CR10_nom)

In [ ]:
# convert to bool
combined_df['TriggerWeight'] = combined_df['TriggerWeight'].astype(bool)

In [ ]:
import pyarrow.feather as feather
feather.write_feather(combined_df, 'data/combined_{}.feather'.format(era))

In [ ]:
combined_df.to_parquet('data/combined_{}.parquet'.format(era))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bck_df = combined_df[(combined_df.type=='bck')]
no_bff = bck_df[(bck_df.SR1_nom > 0 )]
post_bff = bck_df[(bck_df.SR1_nom ==1)]
from src.plotting_tools.Bins import make_bins
bins=make_bins()

In [ ]:
counts, _  = np.histogram(no_bff.DiLepMass,bins=bins.bin_edges, weights=no_bff.Weight)

In [ ]:
counts_cut, _  = np.histogram(post_bff.DiLepMass,bins=bins.bin_edges, weights=post_bff.Weight)

In [ ]:
plt.plot(bins.calc_bin_centers(), counts)
plt.plot(bins.calc_bin_centers(),counts_cut)


In [ ]:
plt.plot(bins.calc_bin_centers(),counts_cut/counts)
plt.ylim(top=1, bottom=0)